# OpenFermion Fermionic Quantum Emulator 
## Phase I Design Proposal

This document outlines the functionality of the API for the OpenFermion Fermionic Quantum Emulator as defined in the design proposal.

 The code is covered and tested with pytest-cov at 99%. Toggle the print option to see output.

In [1]:
fqeprint = True

### 2.A.2 API for data conversion between OpenFermion's backend and OpenFermion-FQE
The API supports data conversion between native Cirq wavefunctions and the FQE wavefunction for arbitrary particle number and spin configuration.  A linequbit wavefunction result is a normalized numpy array.

In [2]:
import fqe
import numpy
import cirq

nqubit = 4

cirq_wfn = fqe.util.rand_wfn(2**nqubit).astype(numpy.complex64)
wfn_from_cirq = fqe.from_cirq(cirq_wfn, 0.0001)

if fqeprint:
    print('Inital wavefunction is \n {} \n with norm {}'.format(cirq_wfn, numpy.vdot(cirq_wfn, cirq_wfn)))
    wfn_from_cirq.print_wfn()

Inital wavefunction is 
 [ 0.23439972+0.15409526j  0.07397068+0.10505342j  0.02592147+0.03159783j
 -0.07644872+0.0900361j  -0.13695174+0.19873293j  0.04263287-0.01148989j
 -0.1752883 +0.0596593j   0.05162946-0.26295862j  0.08237793-0.06866804j
 -0.60891443+0.06489781j -0.20505708-0.14378467j -0.03242815-0.03235062j
  0.2821122 +0.26116204j  0.02045541+0.1722944j  -0.11678796+0.26813033j
  0.07888187-0.04383138j] 
 with norm (1+0j)

Configuration nelectrons: 0 m_s: 0
Vector : 0
 a'00'b'00' : (0.234399721026+0.154095262289j)

Configuration nelectrons: 1 m_s: -1
Vector : 0
 a'00'b'01' : (-0.136951744556+0.198732927442j)
 a'00'b'10' : (0.073970682919+0.105053424835j)

Configuration nelectrons: 1 m_s: 1
Vector : 0
 a'01'b'00' : (0.0823779255152-0.068668037653j)
 a'10'b'00' : (0.025921465829+0.0315978266299j)

Configuration nelectrons: 2 m_s: -2
Vector : 0
 a'00'b'11' : (0.0426328666508-0.0114898867905j)

Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'01'b'01' : (0.282112210989+0.261162042

### 2.A.3 Initialization of wave functions
FQE wave functions can be initialized by calling the constructor directly. fqe.get_wavefunction and get_wavefunction_mutltiple are utility wrappers that provide wrappers for common initialization motifs. e.g. get_spin_nonconserving_wavefunction will build a wavefunction that permits spin breaking Operators to be applied while get_wavefunction_multiple will return a list of initialize wavefunctions

In [3]:
wfFqe = fqe.Wavefunction([[4, -2, 10]])
wfFqe = fqe.get_wavefunction(4, -2, 10)

In [4]:
n = 3
m = -2
wfFQE = fqe.Wavefunction([[n, m, 10], [n+1, m+2, 10]])
wfFQE = fqe.get_wavefunction_multiple([[4, 0, 10], [5, -5, 10]])

In [5]:
wfFQE = fqe.get_spin_nonconserving_wavefunction(2)
wfFQE.set_wfn(strategy='random')
if fqeprint:
    wfFQE.print_wfn()


Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'0001'b'0001' : (-0.148295283318+0.0437329933047j)
 a'0001'b'0010' : (0.0273797865957+0.037615083158j)
 a'0001'b'0100' : (0.0785772502422-0.0999710485339j)
 a'0001'b'1000' : (0.0114503055811+0.217570155859j)
 a'0010'b'0001' : (0.228409469128-0.0488925948739j)
 a'0010'b'0010' : (0.142131119967-0.227016121149j)
 a'0010'b'0100' : (0.0597130656242-0.160798504949j)
 a'0010'b'1000' : (-0.237106248736-0.286548763514j)
 a'0100'b'0001' : (-0.130181461573-0.185091271996j)
 a'0100'b'0010' : (-0.0838004797697+0.343452215195j)
 a'0100'b'0100' : (0.15178373456+0.0414028204978j)
 a'0100'b'1000' : (-0.0448152311146-0.29554054141j)
 a'1000'b'0001' : (-0.157701179385-0.0518541410565j)
 a'1000'b'0010' : (0.268910378218+0.052384249866j)
 a'1000'b'0100' : (-0.0640553385019-0.38268700242j)
 a'1000'b'1000' : (-0.248256370425+0.105605125427j)


### 2.A.4 Element access

Element access is modified from the original propsal to add more convient getter and setter functionalities.  The orginally proposed methods are availible with an addition function to add a value to an existing element.

In [6]:
wfFQE.set_ele(1, 8, .14159265359j)
wfFQE.add_ele(1, 8, 3.j)
imaginary_pi = wfFQE.get_ele(1, 8)
if fqeprint:
    print(-imaginary_pi*1.j)

(3.1415927410125732-0j)


The new components not originally in the proposal allow access of entire vectors for getting and setting methods.

In [7]:
interesting_states = wfn_from_cirq.get_coeff((2, 0), vec=[0,2])
if fqeprint:
    print(interesting_states)

[[ 0.2821122 +0.26116204j  0.        +0.j        ]
 [-0.60891443+0.06489781j  0.        +0.j        ]
 [ 0.1752883 -0.0596593j   0.        +0.j        ]
 [-0.07644872+0.0900361j   0.        +0.j        ]]


In [8]:
interesting_states[:, 1] += 1. + .1j
interesting_states *= .7 - .3j
if fqeprint:
    print(interesting_states)

[[ 0.27582717+0.09817976j  0.73      -0.23000002j]
 [-0.40677074+0.2281028j   0.73      -0.23000002j]
 [ 0.10480402-0.094348j    0.73      -0.23000002j]
 [-0.02650327+0.08595989j  0.73      -0.23000002j]]


In [9]:
data = {}
data[(2, 0)] = interesting_states
wfn_from_cirq.set_wfn(vrange=[0, 2], strategy='from_data', raw_data=data)
if fqeprint:
    wfn_from_cirq.print_wfn()


Configuration nelectrons: 0 m_s: 0
Vector : 0
 a'00'b'00' : (0.234399721026+0.154095262289j)

Configuration nelectrons: 1 m_s: -1
Vector : 0
 a'00'b'01' : (-0.136951744556+0.198732927442j)
 a'00'b'10' : (0.073970682919+0.105053424835j)

Configuration nelectrons: 1 m_s: 1
Vector : 0
 a'01'b'00' : (0.0823779255152-0.068668037653j)
 a'10'b'00' : (0.025921465829+0.0315978266299j)

Configuration nelectrons: 2 m_s: -2
Vector : 0
 a'00'b'11' : (0.0426328666508-0.0114898867905j)

Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'01'b'01' : (0.275827169418+0.0981797575951j)
 a'01'b'10' : (-0.406770735979+0.22810280323j)
 a'10'b'01' : (0.10480401665-0.0943479984999j)
 a'10'b'10' : (-0.0265032686293+0.0859598889947j)

Configuration nelectrons: 2 m_s: 2
Vector : 0
 a'11'b'00' : (-0.20505708456-0.143784672022j)

Configuration nelectrons: 3 m_s: -1
Vector : 0
 a'01'b'11' : (0.0204554144293+0.172294393182j)
 a'10'b'11' : (-0.0516294576228+0.262958616018j)

Configuration nelectrons: 3 m_s: 1
Vector : 

### 2.A.5 Printing/Saving

Printing is currently availible as alpha beta strings followed by the coefficient as well as orbital occupation representation.

In [10]:
print('String forrmatting')
wfn_from_cirq.print_wfn(fmt='str')
print('\n Occupation forrmatting')
wfn_from_cirq.print_wfn(fmt='occ')

String forrmatting

Configuration nelectrons: 0 m_s: 0
Vector : 0
 a'00'b'00' : (0.234399721026+0.154095262289j)

Configuration nelectrons: 1 m_s: -1
Vector : 0
 a'00'b'01' : (-0.136951744556+0.198732927442j)
 a'00'b'10' : (0.073970682919+0.105053424835j)

Configuration nelectrons: 1 m_s: 1
Vector : 0
 a'01'b'00' : (0.0823779255152-0.068668037653j)
 a'10'b'00' : (0.025921465829+0.0315978266299j)

Configuration nelectrons: 2 m_s: -2
Vector : 0
 a'00'b'11' : (0.0426328666508-0.0114898867905j)

Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'01'b'01' : (0.275827169418+0.0981797575951j)
 a'01'b'10' : (-0.406770735979+0.22810280323j)
 a'10'b'01' : (0.10480401665-0.0943479984999j)
 a'10'b'10' : (-0.0265032686293+0.0859598889947j)

Configuration nelectrons: 2 m_s: 2
Vector : 0
 a'11'b'00' : (-0.20505708456-0.143784672022j)

Configuration nelectrons: 3 m_s: -1
Vector : 0
 a'01'b'11' : (0.0204554144293+0.172294393182j)
 a'10'b'11' : (-0.0516294576228+0.262958616018j)

Configuration nelectrons:

### 2.A.6 Fermionic algebra operations and their unitaries on the state

FermionOperators can be directly passed in to create a newwavefunction based on application of the operators.

In [11]:
from openfermion import FermionOperator
ops = FermionOperator('2^ 0', 1.2)
new_wfn = wfn_from_cirq.apply(ops)

In [12]:
new_wfn = fqe.apply(FermionOperator('0^ 0', 1.0), wfFQE)
if fqeprint:
    wfFQE.print_wfn()
    new_wfn.print_wfn()


Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'0001'b'0001' : (-0.148295283318+0.0437329933047j)
 a'0001'b'0010' : (0.0273797865957+0.037615083158j)
 a'0001'b'0100' : (0.0785772502422-0.0999710485339j)
 a'0001'b'1000' : 3.14159274101j
 a'0010'b'0001' : (0.228409469128-0.0488925948739j)
 a'0010'b'0010' : (0.142131119967-0.227016121149j)
 a'0010'b'0100' : (0.0597130656242-0.160798504949j)
 a'0010'b'1000' : (-0.237106248736-0.286548763514j)
 a'0100'b'0001' : (-0.130181461573-0.185091271996j)
 a'0100'b'0010' : (-0.0838004797697+0.343452215195j)
 a'0100'b'0100' : (0.15178373456+0.0414028204978j)
 a'0100'b'1000' : (-0.0448152311146-0.29554054141j)
 a'1000'b'0001' : (-0.157701179385-0.0518541410565j)
 a'1000'b'0010' : (0.268910378218+0.052384249866j)
 a'1000'b'0100' : (-0.0640553385019-0.38268700242j)
 a'1000'b'1000' : (-0.248256370425+0.105605125427j)

Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'0001'b'0001' : (-0.148295283318+0.0437329933047j)
 a'0001'b'0010' : (0.0273797865957+0.03

The fqe has vdot functionality to get an inner product and provides a way to do completely general expectation values.

In [13]:
ops = FermionOperator('2^ 2', 0.5)
opexpec = fqe.vdot(wfFQE, wfFQE.apply(ops))
if fqeprint:
    print(opexpec)

(0.14702515304088593+0j)


In [14]:
ops = FermionOperator('2^ 2', 0.5)
opexpec = fqe.vdot(wfFQE, fqe.apply(ops, wfFQE))
if fqeprint:
    print(opexpec)

(0.14702515304088593+0j)


We can also do general unitary application of operators.  One slight modification to the proposal is that the operator passed in should Hermetian and not anti-Hermitian as stated before.

In [15]:
ops = FermionOperator('2^ 0', 0.2 - .3j) + FermionOperator('0^ 2', 0.2 + .3j)
wfFQE.print_wfn()
wfFQE2 = wfFQE.apply_generated_unitary(ops, algo='taylor')
wfFQE2.print_wfn()
ops = FermionOperator('2^ 0', 0.2 - .3j) + FermionOperator('0^ 2', 0.2 + .3j)
wfFQE2 = fqe.apply_generated_unitary(ops, wfFQE, algo='taylor')
wfFQE2.print_wfn()


Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'0001'b'0001' : (-0.148295283318+0.0437329933047j)
 a'0001'b'0010' : (0.0273797865957+0.037615083158j)
 a'0001'b'0100' : (0.0785772502422-0.0999710485339j)
 a'0001'b'1000' : 3.14159274101j
 a'0010'b'0001' : (0.228409469128-0.0488925948739j)
 a'0010'b'0010' : (0.142131119967-0.227016121149j)
 a'0010'b'0100' : (0.0597130656242-0.160798504949j)
 a'0010'b'1000' : (-0.237106248736-0.286548763514j)
 a'0100'b'0001' : (-0.130181461573-0.185091271996j)
 a'0100'b'0010' : (-0.0838004797697+0.343452215195j)
 a'0100'b'0100' : (0.15178373456+0.0414028204978j)
 a'0100'b'1000' : (-0.0448152311146-0.29554054141j)
 a'1000'b'0001' : (-0.157701179385-0.0518541410565j)
 a'1000'b'0010' : (0.268910378218+0.052384249866j)
 a'1000'b'0100' : (-0.0640553385019-0.38268700242j)
 a'1000'b'1000' : (-0.248256370425+0.105605125427j)

Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'0001'b'0001' : (-0.0247074980289-0.00551100308076j)
 a'0001'b'0010' : (0.00696564931422-0

### 3.B.1 Hamiltonian data structure

Hamiltonians can be incorporated from OpenFermion objects or initialized from numpy arrays.  Notice a slight change in the arguments to get_two_body to include a constant potential term and additionally a chemical potential to be consistent with OpenFermion.

In [16]:
from openfermion import get_quadratic_hamiltonian
ops = FermionOperator((), 1.) + FermionOperator('1^ 1', 3.) + FermionOperator('1^ 2', 3. + 4.j) + FermionOperator('2^ 1', 3. - 4.j) + FermionOperator('3^ 4', 2. + 5.j) + FermionOperator('4^ 3', 2. - 5.j)
hamOF = get_quadratic_hamiltonian(ops, -0.03)
hamFQE = fqe.get_hamiltonian_from_openfermion(hamOF)
hamFQE = fqe.get_quadratic_hamiltonian(ops, -0.03)

Symmetry is passed in by lists containing permutation operations, a phase and whether or not complex conjugation is used.  Below are examples of a Hermetian matrix and a real anti-symmetric matrix.

In [17]:
e = 1.0
t = numpy.random.rand(4, 4) + numpy.random.rand(4, 4)*1.j
h = t + numpy.conjugate(t.T)
symmh = [
    [[1, 2], 1.0, False],
    [[2, 1], 1.0, True]
]
t = numpy.random.rand(4, 4, 4, 4)
g = t - t.T
symmg = [
    [[1, 2, 3, 4], 1.0, True],
    [[4, 3, 2, 1], -1.0, True]
]
hamFQE = fqe.get_two_body_hamiltonian(-0.3, h, g, 0.0, symmh, symmg)

The FQE will also return PolynomialTensor objects for parsing by OpenFermion objects.

In [18]:
hamOF = fqe.hamiltonian_to_openfermion(hamFQE)